In [ ]:
from dotenv import load_dotenv
load_dotenv(override=True)
import openai
import os
openai.api_key = os.environ["OPENAI_API_KEY"]

# Embeddings

In [ ]:
from openai.embeddings_utils import cosine_similarity

response = openai.Embedding.create(
    input="Kobe Bryant",
    model="text-embedding-ada-002"
)

embedding1 = response['data'][0]['embedding']

response = openai.Embedding.create(
    input="Micheal Jordan",
    model="text-embedding-ada-002"
)

embedding2 = response['data'][0]['embedding']

cosine_similarity(embedding1, embedding2)

# Fine-tuning

In [ ]:
from dotenv import load_dotenv
load_dotenv(override=True)
import os
print(os.environ["OPENAI_API_KEY"])
from os.path import join
import json
import pandas as pd
import random
import numpy as np
import sys
sys.path.append("../..")
from column_annotation_gnn.data_loader.SportsDB_data_loader import get_LabelEncoder

label_enc = get_LabelEncoder()

## Data Preparation

### Sport-Domains / Tablewise

In [ ]:
# Generating training data for gpt
sport_domains = ["baseball", "basketball", "football", "hockey", "soccer"]
#sport_domains = ["baseball"]
shuffle_cols = False
random_state = 1
split = "train"
number_of_rows_per_table = 10
end_of_prompt = "\n\n###\n\n"

for random_state in range(1,6):
    for sport_domain in sport_domains:
        training_data_prompts = []
        all_class_labels = []
        # load metadata.json containing semantic types of the columns
        with open(join(os.environ["SportsTables"], sport_domain, "metadata.json")) as f:
            metadata = json.load(f)
        with open(join(os.environ["SportsTables"], sport_domain, f"train_valid_test_split_{random_state}.json")) as f:
                    train_valid_test_split = json.load(f)
                    
        for idx_table_path, table_name_full in enumerate(train_valid_test_split[split]):
            training_data = {}
            # if idx_table_path > 0:
            #     continue
            table_name = table_name_full.split("/")[-1].split(".csv")[0]
            ## search for correct in key in metadata
            table_metadata_key = None
            for key in metadata.keys():
                if key in table_name:
                    table_metadata_key = key
            if table_metadata_key == None:
                print(f"CSV {table_name_full} not in metadata.json defined!")
                continue

            df = pd.read_csv(join(os.environ["SportsTables"], sport_domain, table_name_full))
            valid_cols = []
            valid_labels = []
            if shuffle_cols:
                column_list = list(range(len(df.columns)))
                #random.seed(self.random_state)
                random.shuffle(column_list)
            else:
                column_list = list(range(len(df.columns)))
                
            for i in column_list:
                column_name = df.columns[i]
                # search for defined columns data type and semantic label in metadata
                if column_name in metadata[table_metadata_key]["textual_cols"].keys():
                    column_data_type = "textual"
                    column_label = metadata[table_metadata_key]["textual_cols"][column_name]
                elif column_name in metadata[table_metadata_key]["numerical_cols"].keys():
                    column_data_type = "numerical"
                    column_label = metadata[table_metadata_key]["numerical_cols"][column_name]
                else:
                    print(f"Column {df.columns[i]} in {table_name} not labeled in metadata.json!")
                    continue
                
                if column_label == None:
                    print(f"Column {df.columns[i]} in {table_name} not labeled in metadata.json!")
                    continue
                
                valid_cols.append(column_name)
                valid_labels.append(column_label)

            df_result = df[valid_cols][:number_of_rows_per_table]
            if len(df_result) == 0:
                print(f"Table {table_name} ({idx_table_path}) has no columns with assigned semantic types!")
                continue
            
            try:
                training_data["prompt"] = df_result.to_csv(header=False)+end_of_prompt
                training_data["completion"] = " "+"\n".join(valid_labels)
                training_data_prompts.append(training_data)
                all_class_labels.extend(valid_labels)
            except Exception as e:
                print(e)
                print(valid_labels)
                
        with open(f"./training_data/{sport_domain}_{split}_{shuffle_cols}_{random_state}.jsonl", "w") as f:
            for entry in training_data_prompts:
                json.dump(entry, f)
                f.write("\n")
        print(f"{sport_domain}_{split}_{shuffle_cols}_{random_state}.jsonl Number of unique classes: {len(np.unique(all_class_labels))}")

In [ ]:
# Generating training data for gpt 
# Overall sport datasets

sport_domains = ["baseball", "basketball", "football", "hockey", "soccer"]
shuffle_cols = True
random_state = 1
split = "test"
number_of_rows_per_table = 10
end_of_prompt = "\n\n###\n\n"

for random_state in range(1,6):
    training_data_prompts = []
    all_class_labels = []
    for sport_domain in sport_domains:
        # load metadata.json containing semantic types of the columns
        with open(join(os.environ["SportsTables"], sport_domain, "metadata.json")) as f:
            metadata = json.load(f)
        with open(join(os.environ["SportsTables"], sport_domain, f"train_valid_test_split_{random_state}.json")) as f:
                    train_valid_test_split = json.load(f)
                    
        for idx_table_path, table_name_full in enumerate(train_valid_test_split[split]):
            training_data = {}
            # if idx_table_path > 2:
            #     continue
            table_name = table_name_full.split("/")[-1].split(".csv")[0]
            ## search for correct in key in metadata
            table_metadata_key = None
            for key in metadata.keys():
                if key in table_name:
                    table_metadata_key = key
            if table_metadata_key == None:
                #print(f"CSV {table_name_full} not in metadata.json defined!")
                continue

            df = pd.read_csv(join(os.environ["SportsTables"], sport_domain, table_name_full))
            valid_cols = []
            valid_labels = []
            if shuffle_cols:
                column_list = list(range(len(df.columns)))
                #random.seed(self.random_state)
                random.shuffle(column_list)
            else:
                column_list = list(range(len(df.columns)))
                
            for i in column_list:
                column_name = df.columns[i]
                # search for defined columns data type and semantic label in metadata
                if column_name in metadata[table_metadata_key]["textual_cols"].keys():
                    column_data_type = "textual"
                    column_label = metadata[table_metadata_key]["textual_cols"][column_name]
                elif column_name in metadata[table_metadata_key]["numerical_cols"].keys():
                    column_data_type = "numerical"
                    column_label = metadata[table_metadata_key]["numerical_cols"][column_name]
                else:
                    #print(f"Column {df.columns[i]} in {table_name} not labeled in metadata.json!")
                    continue
                
                if column_label == None:
                    #print(f"Column {df.columns[i]} in {table_name} not labeled in metadata.json!")
                    continue
                
                valid_cols.append(column_name)
                valid_labels.append(column_label)
            
            df_result = df[valid_cols][:number_of_rows_per_table]
            if len(df_result) == 0:
                #print(f"Table {table_name} has no columns with assigned semantic types!")
                continue
            
            try:
                valid_labels = label_enc.transform(valid_labels)
                training_data["prompt"] = df_result.to_csv(header=False)+end_of_prompt
                training_data["completion"] = " "+"\n".join([str(x) for x in valid_labels])
                training_data_prompts.append(training_data)
                all_class_labels.extend(valid_labels)
            except Exception as e:
                print(e)
                print(valid_labels)
                
    with open(f"./training_data/overall_{split}_{shuffle_cols}_{random_state}.jsonl", "w") as f:
        for entry in training_data_prompts:
            json.dump(entry, f)
            f.write("\n")
    print(f"overall_{split}_{shuffle_cols}_{random_state}.jsonl Number of unique classes: {len(np.unique(all_class_labels))}")

### Overall / Tablewise transposed

In [ ]:
# Generating training data for gpt 
# Overall sport datasets
# Now in the form of one row belongs to one column => transposed table

sport_domains = ["baseball", "basketball", "football", "hockey", "soccer"]
shuffle_cols = True
random_state = 1
split = "test"
number_of_rows_per_table = 10
end_of_prompt = "\n\n###\n\n"

for random_state in range(1,6):
    training_data_prompts = []
    all_class_labels = []
    for sport_domain in sport_domains:
        # load metadata.json containing semantic types of the columns
        with open(join(os.environ["SportsTables"], sport_domain, "metadata.json")) as f:
            metadata = json.load(f)
        with open(join(os.environ["SportsTables"], sport_domain, f"train_valid_test_split_{random_state}.json")) as f:
                    train_valid_test_split = json.load(f)
                    
        for idx_table_path, table_name_full in enumerate(train_valid_test_split[split]):
            training_data = {}
            # if idx_table_path > 250:
            #     continue
            table_name = table_name_full.split("/")[-1].split(".csv")[0]
            ## search for correct in key in metadata
            table_metadata_key = None
            for key in metadata.keys():
                if key in table_name:
                    table_metadata_key = key
            if table_metadata_key == None:
                #print(f"CSV {table_name_full} not in metadata.json defined!")
                continue

            df = pd.read_csv(join(os.environ["SportsTables"], sport_domain, table_name_full))
            valid_cols = []
            valid_labels = []
            if shuffle_cols:
                column_list = list(range(len(df.columns)))
                #random.seed(self.random_state)
                random.shuffle(column_list)
            else:
                column_list = list(range(len(df.columns)))
                
            for i in column_list:
                column_name = df.columns[i]
                # search for defined columns data type and semantic label in metadata
                if column_name in metadata[table_metadata_key]["textual_cols"].keys():
                    column_data_type = "textual"
                    column_label = metadata[table_metadata_key]["textual_cols"][column_name]
                elif column_name in metadata[table_metadata_key]["numerical_cols"].keys():
                    column_data_type = "numerical"
                    column_label = metadata[table_metadata_key]["numerical_cols"][column_name]
                else:
                    #print(f"Column {df.columns[i]} in {table_name} not labeled in metadata.json!")
                    continue
                
                if column_label == None:
                    #print(f"Column {df.columns[i]} in {table_name} not labeled in metadata.json!")
                    continue
                
                valid_cols.append(column_name)
                valid_labels.append(column_label)
            
            if len(valid_cols) == 0:
                continue
            df_result = df[valid_cols][:number_of_rows_per_table]
            if len(df_result) == 0:
                #print(f"Table {table_name} has no columns with assigned semantic types!")
                continue
            
            try:
                valid_labels = label_enc.transform(valid_labels)
                training_data["prompt"] = df_result.T.to_csv(header=False, index=False)+end_of_prompt
                training_data["completion"] = " "+"\n".join([str(x) for x in valid_labels])
                training_data_prompts.append(training_data)
                all_class_labels.extend(valid_labels)
            except Exception as e:
                print(e)
                print(valid_labels)
                
    with open(f"./training_data/overall_{split}_{shuffle_cols}_{random_state}_transposed.jsonl", "w") as f:
        for entry in training_data_prompts:
            json.dump(entry, f)
            f.write("\n")
    print(f"overall_{split}_{shuffle_cols}_{random_state}.jsonl Number of unique classes: {len(np.unique(all_class_labels))}")

### Overall / Columnwise 

In [ ]:
# Generating training data for gpt 
# Overall sport datasets
# Now in the form of one row belongs to one column => transposed table
# Now in the form that one prompt/completion pair belongs to one table-column

sport_domains = ["baseball", "basketball", "football", "hockey", "soccer"]
shuffle_cols = True
random_state = 1
split = "test"
number_of_rows_per_table = 10
end_of_prompt = "\n\n###\n\n"

for random_state in range(1,6):
    training_data_prompts = []
    all_class_labels = []
    for sport_domain in sport_domains:
        # load metadata.json containing semantic types of the columns
        with open(join(os.environ["SportsTables"], sport_domain, "metadata.json")) as f:
            metadata = json.load(f)
        with open(join(os.environ["SportsTables"], sport_domain, f"train_valid_test_split_{random_state}.json")) as f:
                    train_valid_test_split = json.load(f)
                    
        for idx_table_path, table_name_full in enumerate(train_valid_test_split[split]):
            # if idx_table_path > 0:
            #     continue
            table_name = table_name_full.split("/")[-1].split(".csv")[0]
            ## search for correct in key in metadata
            table_metadata_key = None
            for key in metadata.keys():
                if key in table_name:
                    table_metadata_key = key
            if table_metadata_key == None:
                #print(f"CSV {table_name_full} not in metadata.json defined!")
                continue

            df = pd.read_csv(join(os.environ["SportsTables"], sport_domain, table_name_full))
            valid_cols = []
            valid_labels = []
            if shuffle_cols:
                column_list = list(range(len(df.columns)))
                #random.seed(self.random_state)
                random.shuffle(column_list)
            else:
                column_list = list(range(len(df.columns)))
                
            for i in column_list:
                column_name = df.columns[i]
                # search for defined columns data type and semantic label in metadata
                if column_name in metadata[table_metadata_key]["textual_cols"].keys():
                    column_data_type = "textual"
                    column_label = metadata[table_metadata_key]["textual_cols"][column_name]
                elif column_name in metadata[table_metadata_key]["numerical_cols"].keys():
                    column_data_type = "numerical"
                    column_label = metadata[table_metadata_key]["numerical_cols"][column_name]
                else:
                    #print(f"Column {df.columns[i]} in {table_name} not labeled in metadata.json!")
                    continue
                
                if column_label == None:
                    #print(f"Column {df.columns[i]} in {table_name} not labeled in metadata.json!")
                    continue
                
                valid_cols.append(column_name)
                valid_labels.append(column_label)
            
            if len(valid_cols) == 0:
                continue
            df_result = df[valid_cols][:number_of_rows_per_table]
            if len(df_result) == 0:
                #print(f"Table {table_name} has no columns with assigned semantic types!")
                continue
            
            try:
                for idx, column in enumerate(df_result.columns):
                    training_data = {}
                    training_data["prompt"] = df_result[column].to_csv(header=False, index=False)+end_of_prompt
                    training_data["completion"] = " "+str(label_enc.transform([valid_labels[idx]])[0])
                    training_data_prompts.append(training_data)
                    all_class_labels.append(valid_labels[idx])
            except Exception as e:
                print(e)
                print(valid_labels)
                
    with open(f"./training_data/overall_{split}_{shuffle_cols}_{random_state}_columnwise.jsonl", "w") as f:
        for entry in training_data_prompts:
            json.dump(entry, f)
            f.write("\n")
    print(f"overall_{split}_{shuffle_cols}_{random_state}.jsonl Number of unique classes: {len(np.unique(all_class_labels))}")

In [ ]:
!openai tools fine_tunes.prepare_data -f ./training_data/overall_train_False_1.jsonl

In [ ]:
!openai api fine_tunes.create -t ./training_data/GitTables_train_True_1_0.7_columnwise.jsonl -m ada --suffix GitTables-true-1-0.7-columnwise

## Using the model

In [ ]:
from dotenv import load_dotenv
load_dotenv(override=True)
import os
import openai
openai.api_key = os.environ["OPENAI_API_KEY"]
import pandas as pd
from sklearn.metrics import classification_report
from tqdm import tqdm
import json

random_state = 1
shuffle_cols = True
sport_domain = "overall"
modifications = "transposed"

# load test data
if modifications == None:
    test_dataset = pd.read_json(f"./training_data/{sport_domain}_test_{shuffle_cols}_{random_state}.jsonl", lines=True)
else:
    test_dataset = pd.read_json(f"./training_data/{sport_domain}_test_{shuffle_cols}_{random_state}_{modifications}.jsonl", lines=True)

model_dic = {
    "overall": {
        "False": {
            "1": "ada:ft-personal:overall-false-1-2023-03-27-15-30-00",
            "2": "ada:ft-personal:overall-false-2-2023-03-27-17-27-58"
        },
        "True": {
            "1": "ada:ft-data-and-ai-systems-tu-darmstadt:overall-true-1-2023-04-01-22-50-33",
            "2": "ada:ft-data-and-ai-systems-tu-darmstadt:overall-true-2-2023-03-28-08-47-27"
        }
    },
    "overall_columnwise":{
        "False":{
            "1":"ada:ft-data-and-ai-systems-tu-darmstadt:overall-train-false-columnwisee-2023-03-30-21-05-01"
        }
    },
    "overall_transposed":{
        "True":{
            "1":"ada:ft-data-and-ai-systems-tu-darmstadt:overall-true-1-transposed-2023-04-02-19-57-41"
        }
    },
    "baseball": {
        "False": {
            "1":"ada:ft-data-and-ai-systems-tu-darmstadt:baseball-false-1-2023-03-28-22-19-35"
            },
        "True": {
            "1":"ada:ft-data-and-ai-systems-tu-darmstadt:baseball-true-1-2023-03-29-00-11-28"
            }
    }
}

# load model
if modifications == None:
    model = model_dic[sport_domain][str(shuffle_cols)][str(random_state)]
else:
    model = model_dic[f"{sport_domain}_{modifications}"][str(shuffle_cols)][str(random_state)]
print(f"Loaded model: {model}")  

# test model
total_true_labels = []
total_predicted_labels = []
for i in tqdm(range(0,len(test_dataset))):    
    response = openai.Completion.create(
        model= model,
        temperature = 0.8,
        max_tokens=600,
        prompt= test_dataset.iloc[i]["prompt"]
    )
    number_of_predictions = len(test_dataset.iloc[i]["completion"].split("\n"))
    true_labels = test_dataset.iloc[i]["completion"].split("\n")
    true_labels[0] = true_labels[0].replace(" ", "")
    
    predicted_labels = response["choices"][0]["text"].split("\n")[:number_of_predictions]
    predicted_labels[0] = predicted_labels[0].replace(" ", "")
    
    if number_of_predictions != len(predicted_labels):
        print(f"Number of labels should be {number_of_predictions} but is {len(predicted_labels)}")
        print(i)
        break
        
    total_true_labels.extend(true_labels)
    total_predicted_labels.extend(predicted_labels)
    
class_report = classification_report(
    total_true_labels, total_predicted_labels, output_dict=True)

with open(f"./results/classification_report_{sport_domain}_{shuffle_cols}_{random_state}_{modifications}.json", "w") as f:
    json.dump(class_report, f)
    
with open(f"./results/predictions_{sport_domain}_{shuffle_cols}_{random_state}_{modifications}.json", "w") as f:
    json.dump({"y_pred":total_predicted_labels, "y_true":total_true_labels}, f)

In [ ]:
## Test columnwise model

from dotenv import load_dotenv
load_dotenv(override=True)
import os
import openai
openai.api_key = os.environ["OPENAI_API_KEY"]
import pandas as pd
from sklearn.metrics import classification_report
from tqdm import tqdm
import json

random_state = 5
shuffle_cols = True
sport_domain = "overall"
modifications = "columnwise"

# load test data
if modifications == None:
    test_dataset = pd.read_json(f"./training_data/{sport_domain}_test_{shuffle_cols}_{random_state}.jsonl", lines=True)
else:
    test_dataset = pd.read_json(f"./training_data/{sport_domain}_test_{shuffle_cols}_{random_state}_{modifications}.jsonl", lines=True)

model_dic = {
    "overall": {
        "False": {
            "1": "ada:ft-personal:overall-false-1-2023-03-27-15-30-00",
            "2": "ada:ft-personal:overall-false-2-2023-03-27-17-27-58"
        },
        "True": {
            "1": "ada:ft-personal:overall-true-1-2023-03-27-15-56-42",
            "2": "ada:ft-data-and-ai-systems-tu-darmstadt:overall-true-2-2023-03-28-08-47-27"
        }
    },
    "overall_columnwise":{
        "False":{
            "1":"ada:ft-data-and-ai-systems-tu-darmstadt:overall-train-false-columnwisee-2023-03-30-21-05-01"
        },
        "True":{
            "1":"ada:ft-data-and-ai-systems-tu-darmstadt:overall-true-1-columnwise-2023-04-03-13-31-50",
            "2":"ada:ft-data-and-ai-systems-tu-darmstadt:overall-true-2-columnwise-2023-04-01-22-43-04",
            "3":"ada:ft-data-and-ai-systems-tu-darmstadt:overall-true-3-columnwise-2023-06-10-01-13-49",
            "4":"ada:ft-data-and-ai-systems-tu-darmstadt:overall-true-4-columnwise-2023-06-10-01-39-44",
            "5":"ada:ft-data-and-ai-systems-tu-darmstadt:overall-true-5-columnwise-2023-06-10-02-05-42"
        }
    },
    "overall_transposed":{
        "True":{
            "1":"ada:ft-data-and-ai-systems-tu-darmstadt:overall-true-1-transposed-2023-03-30-20-13-28"
        }
    },
    "baseball": {
        "False": {
            "1":"ada:ft-data-and-ai-systems-tu-darmstadt:baseball-false-1-2023-03-28-22-19-35"
            },
        "True": {
            "1":"ada:ft-data-and-ai-systems-tu-darmstadt:baseball-true-1-2023-03-29-00-11-28"
            }
    }
}

# load model
if modifications == None:
    model = model_dic[sport_domain][str(shuffle_cols)][str(random_state)]
else:
    model = model_dic[f"{sport_domain}_{modifications}"][str(shuffle_cols)][str(random_state)]
print(f"Loaded model: {model}")  

# test model
total_true_labels = []
total_predicted_labels = []
for i in tqdm(range(0,len(test_dataset))):    
    response = openai.Completion.create(
        model= model,
        temperature = 0.8,
        max_tokens=1,
        prompt= test_dataset.iloc[i]["prompt"]
    )
    #number_char_of_predictions = len(test_dataset.iloc[i]["completion"])
    true_label = str(test_dataset.iloc[i]["completion"])
    
    #predicted_label = response["choices"][0]["text"][:number_char_of_predictions]
    predicted_label = response["choices"][0]["text"].replace(" ", "")
        
    total_true_labels.append(true_label)
    total_predicted_labels.append(predicted_label)
    
class_report = classification_report(
    total_true_labels, total_predicted_labels, output_dict=True)

with open(f"./results/classification_report_{sport_domain}_{shuffle_cols}_{random_state}_{modifications}.json", "w") as f:
    json.dump(class_report, f)
    
with open(f"./results/predictions_{sport_domain}_{shuffle_cols}_{random_state}_{modifications}.json", "w") as f:
    json.dump({"y_pred":total_predicted_labels, "y_true":total_true_labels}, f)

In [ ]:
from dotenv import load_dotenv
load_dotenv(override=True)
import os
import openai
openai.api_key = os.environ["OPENAI_API_KEY"]
import pandas as pd
from sklearn.metrics import classification_report
from tqdm import tqdm
import json

random_state = 1
shuffle_cols = True
sport_domain = "overall"
modifications = "columnwise"

# load test data
if modifications == None:
    test_dataset = pd.read_json(f"./training_data/{sport_domain}_test_{shuffle_cols}_{random_state}.jsonl", lines=True)
else:
    test_dataset = pd.read_json(f"./training_data/{sport_domain}_test_{shuffle_cols}_{random_state}_{modifications}.jsonl", lines=True)

model_dic = {
    "overall": {
        "False": {
            "1": "ada:ft-personal:overall-false-1-2023-03-27-15-30-00",
            "2": "ada:ft-personal:overall-false-2-2023-03-27-17-27-58"
        },
        "True": {
            "1": "ada:ft-personal:overall-true-1-2023-03-27-15-56-42",
            "2": "ada:ft-data-and-ai-systems-tu-darmstadt:overall-true-2-2023-03-28-08-47-27"
        }
    },
    "overall_columnwise":{
        "False":{
            "1":"ada:ft-data-and-ai-systems-tu-darmstadt:overall-train-false-columnwisee-2023-03-30-21-05-01"
        },
        "True":{
            "1": "ada:ft-data-and-ai-systems-tu-darmstadt:overall-true-1-columnwise-2023-04-01-21-14-53"
        }
    },
    "overall_transposed":{
        "True":{
            "1":"ada:ft-data-and-ai-systems-tu-darmstadt:overall-true-1-transposed-2023-03-30-20-13-28"
        }
    },
    "baseball": {
        "False": {
            "1":"ada:ft-data-and-ai-systems-tu-darmstadt:baseball-false-1-2023-03-28-22-19-35"
            },
        "True": {
            "1":"ada:ft-data-and-ai-systems-tu-darmstadt:baseball-true-1-2023-03-29-00-11-28"
            }
    }
}

# load model
if modifications == None:
    model = model_dic[sport_domain][str(shuffle_cols)][str(random_state)]
else:
    model = model_dic[f"{sport_domain}_{modifications}"][str(shuffle_cols)][str(random_state)]
print(f"Loaded model: {model}")  

In [ ]:
type(test_dataset.iloc[27]["completion"])

In [ ]:
len(test_dataset.iloc[0]["completion"])

In [ ]:
response = openai.Completion.create(
        model= model,
        temperature = 0.8,
        max_tokens=1,
        prompt= test_dataset.iloc[27]["prompt"]
    )

In [ ]:
int(response["choices"][0]["text"].replace(" ", ""))

In [ ]:
label_enc.inverse_transform([55])

## GitTables

### Data preparation

In [ ]:
from os.path import join
import pyarrow.parquet as pq
import random
import pandas as pd
import numpy as np
import json
import sys
sys.path.append("../..")
from tqdm import tqdm
from column_annotation_gnn.data_loader.GitTables_data_loader import get_LabelEncoder

label_enc = get_LabelEncoder()

# tablewise
shuffle_cols = True
random_state = 1
split = "test"
number_of_rows_per_table = 10
end_of_prompt = "\n\n###\n\n"

for random_state in range(1,2):
    training_data_prompts = []
    all_class_labels = []
    with open(join(os.environ["GitTables"], "data", f"train_valid_test_split_{random_state}_0.7.json")) as f:
        train_valid_test_split = json.load(f)
                
    for idx_table_path, table_path in tqdm(enumerate(train_valid_test_split[split]), total=len(train_valid_test_split[split])):
        training_data = {}
        # if idx_table_path > 0:
        #     continue
        
        # read metadata
        table_metadata = json.loads(pq.read_schema(join(os.environ["GitTables"], table_path)).metadata[b"gittables"])
        dbpedia_types = table_metadata["dbpedia_embedding_column_types"]
        dbpedia_similarities = table_metadata["dbpedia_embedding_similarities"]

        # read the table in a DF
        df = pd.read_parquet(join(os.environ["GitTables"], table_path))
            
        valid_cols = []
        valid_labels = []
        if shuffle_cols:
            column_list = list(range(len(df.columns)))
            #random.seed(self.random_state)
            random.shuffle(column_list)
        else:
            column_list = list(range(len(df.columns)))
            
        for i in column_list:
            column_name = df.columns[i]
            
            try:
                # check if the semantic type of the columns is in the valid semantic types that we consider in our experiments 
                if len(df[column_name].dropna()) > 0:
                    if dbpedia_similarities[column_name] >= 0.7:
                        if table_metadata["dtypes"][column_name] == "object" or table_metadata["dtypes"][column_name] == "string":
                            if (dbpedia_types[column_name]["cleaned_label"]+"_tt" in label_enc.classes_):
                                column_data_type = 0 # => "textual"
                                column_label = dbpedia_types[column_name]["cleaned_label"]+"_tt"
                            else:
                                 continue
                        else:
                            if (dbpedia_types[column_name]["cleaned_label"]+"_nt" in label_enc.classes_):
                                column_data_type = 1 # => "numerical"
                                column_label = dbpedia_types[column_name]["cleaned_label"]+"_nt"
                            else:
                                continue
                    else:
                        continue
                else:
                    continue
                
            except Exception as e:
                continue
                #print(e)
                #print(f"Not considering column: {column_name} from table: {table_path}")
            
            valid_cols.append(column_name)
            valid_labels.append(column_label)
            
        df_result = df[valid_cols][:number_of_rows_per_table]
        if len(df_result) == 0:
            #print(f"Table {table_name} has no columns with assigned semantic types!")
            continue
        
        try:
            valid_labels = label_enc.transform(valid_labels)
            training_data["prompt"] = df_result.to_csv(header=False)+end_of_prompt
            training_data["completion"] = " "+"\n".join([str(x) for x in valid_labels])
            training_data_prompts.append(training_data)
            all_class_labels.extend(valid_labels)
        except Exception as e:
            print(e)
            print(valid_labels)
                
    with open(f"./training_data/GitTables_{split}_{shuffle_cols}_{random_state}_0.7.jsonl", "w") as f:
        for entry in training_data_prompts:
            json.dump(entry, f)
            f.write("\n")
    print(f"overall_{split}_{shuffle_cols}_{random_state}.jsonl Number of unique classes: {len(np.unique(all_class_labels))}")

In [ ]:
# columnwise
from os.path import join
import pyarrow.parquet as pq
import random
import pandas as pd
import numpy as np
import json
import sys
sys.path.append("../..")
from column_annotation_gnn.data_loader.GitTables_data_loader import get_LabelEncoder

label_enc = get_LabelEncoder()
 
# Generating training data for gpt 
# Overall sport datasets
# Now in the form of one row belongs to one column => transposed table
# Now in the form that one prompt/completion pair belongs to one table-column

shuffle_cols = True
random_state = 1
split = "train"
number_of_rows_per_table = 64
end_of_prompt = "\n\n###\n\n"

for random_state in range(1,6):
    training_data_prompts = []
    all_class_labels = []
    with open(join(os.environ["GitTables"], "data", f"train_valid_test_split_{random_state}.json")) as f:
        train_valid_test_split = json.load(f)
                
    for idx_table_path, table_path in tqdm(enumerate(train_valid_test_split[split], total=len(train_valid_test_split[split]))):
        # if idx_table_path > 0:
        #     continue
        
        # read metadata
        table_metadata = json.loads(pq.read_schema(join(os.environ["GitTables"], table_path)).metadata[b"gittables"])
        dbpedia_types = table_metadata["dbpedia_embedding_column_types"]
        dbpedia_similarities = table_metadata["dbpedia_embedding_similarities"]

        # read the table in a DF
        df = pd.read_parquet(join(os.environ["GitTables"], table_path))
            
        valid_cols = []
        valid_labels = []
        if shuffle_cols:
            column_list = list(range(len(df.columns)))
            #random.seed(self.random_state)
            random.shuffle(column_list)
        else:
            column_list = list(range(len(df.columns)))
            
        for i in column_list:
            column_name = df.columns[i]
            
            try:
                # check if the semantic type of the columns is in the valid semantic types that we consider in our experiments 
                if dbpedia_types[column_name]["cleaned_label"] in label_enc.classes_:
                    # in case we want to filter assigned types regarding the similarity score later on
                    if dbpedia_similarities[column_name] > 0.0: 
                        if table_metadata["dtypes"][column_name] == "object" or table_metadata["dtypes"][column_name] == "string":
                            column_data_type = "textual"
                            column_label = dbpedia_types[column_name]["cleaned_label"]
                        else:
                            column_data_type = "numerical"
                            column_label = dbpedia_types[column_name]["cleaned_label"]
                else:
                    continue
                
            except Exception as e:
                continue
                #print(e)
                
                #print(f"Not considering column: {column_name} from table: {table_path}")
            
            valid_cols.append(column_name)
            valid_labels.append(column_label)
        
        if len(valid_cols) == 0:
            continue
        if len(df_result) >= number_of_rows_per_table:
            df_result = df[valid_cols][:number_of_rows_per_table]
        else:
            df_result = df[valid_cols]
        if len(df_result) == 0:
            #print(f"Table {table_name} has no columns with assigned semantic types!")
            continue
        try:
            for idx, column in enumerate(df_result.columns):
                training_data = {}
                training_data["prompt"] = df_result[column].to_csv(header=False, index=False)+end_of_prompt
                training_data["completion"] = " "+str(label_enc.transform([valid_labels[idx]])[0])
                training_data_prompts.append(training_data)
                all_class_labels.append(valid_labels[idx])
        except Exception as e:
            print(e)
            print(valid_labels)
                
    with open(f"./training_data/GitTables_{split}_{shuffle_cols}_{random_state}_columnwise.jsonl", "w") as f:
        for entry in training_data_prompts:
            json.dump(entry, f)
            f.write("\n")
    print(f"GitTables_{split}_{shuffle_cols}_{random_state}.jsonl Number of unique classes: {len(np.unique(all_class_labels))}")

### Test the model

In [ ]:
## Test columnwise model

from dotenv import load_dotenv
load_dotenv(override=True)
import os
import openai
openai.api_key = os.environ["OPENAI_API_KEY"]
import pandas as pd
from sklearn.metrics import classification_report
from tqdm import tqdm
import json

random_state = 1
shuffle_cols = True
modifications = "columnwise"

# load test data
if modifications == None:
    test_dataset = pd.read_json(f"./training_data/GitTables_test_{shuffle_cols}_{random_state}.jsonl", lines=True)
else:
    test_dataset = pd.read_json(f"./training_data/GitTables_test_{shuffle_cols}_{random_state}_0.7_{modifications}.jsonl", lines=True)

model_dic = {
    "tablewise": {
        "True":{
            "1": "ada:ft-data-and-ai-systems-tu-darmstadt:gittables-true-1-0-7-2023-11-08-11-10-13"
        }
    },
    "columnwise":{
        "True":{
            "1":"ada:ft-data-and-ai-systems-tu-darmstadt:gittables-true-1-0-7-columnwise-2023-11-07-15-24-43",
        }
    }
}

# load model
if modifications == None:
    model = model_dic["tablewise"][str(shuffle_cols)][str(random_state)]
else:
    model = model_dic[f"{modifications}"][str(shuffle_cols)][str(random_state)]
print(f"Loaded model: {model}")  

# test model
total_true_labels = []
total_predicted_labels = []
for i in tqdm(range(0,len(test_dataset))):
    # if i > 0:
    #     break    
    response = openai.Completion.create(
        model= model,
        temperature = 0.8,
        max_tokens=600,
        prompt= test_dataset.iloc[i]["prompt"]
    )
    # print(test_dataset.iloc[i]["prompt"])
    # print(response)
    #number_char_of_predictions = len(test_dataset.iloc[i]["completion"])
    true_label = str(test_dataset.iloc[i]["completion"])
    
    #predicted_label = response["choices"][0]["text"][:number_char_of_predictions]
    predicted_label = response["choices"][0]["text"].replace(" ", "")
        
    total_true_labels.append(true_label)
    total_predicted_labels.append(predicted_label)
    
class_report = classification_report(
    total_true_labels, total_predicted_labels, output_dict=True)

with open(f"./results/GitTables_classification_report_{shuffle_cols}_{random_state}_0.7_{modifications}.json", "w") as f:
    json.dump(class_report, f)
    
with open(f"./results/GitTables_predictions_{shuffle_cols}_{random_state}_0.7_{modifications}.json", "w") as f:
    json.dump({"y_pred":total_predicted_labels, "y_true":total_true_labels}, f)

In [ ]:
## Test tablewise model
from dotenv import load_dotenv
load_dotenv(override=True)
import os
import openai
openai.api_key = os.environ["OPENAI_API_KEY"]
import pandas as pd
from sklearn.metrics import classification_report
from tqdm import tqdm
import json

random_state = 1
shuffle_cols = True
modifications = None

# load test data
if modifications == None:
    test_dataset = pd.read_json(f"./training_data/GitTables_test_{shuffle_cols}_{random_state}_0.7.jsonl", lines=True)
else:
    test_dataset = pd.read_json(f"./training_data/GitTables_test_{shuffle_cols}_{random_state}_0.7_{modifications}.jsonl", lines=True)

model_dic = {
    "tablewise": {
        "True":{
            "1": "ada:ft-data-and-ai-systems-tu-darmstadt:gittables-true-1-0-7-2023-11-08-11-10-13"
        }
    },
    "columnwise":{
        "True":{
            "1":"ada:ft-data-and-ai-systems-tu-darmstadt:gittables-true-1-0-7-columnwise-2023-11-07-15-24-43",
        }
    }
}

# load model
if modifications == None:
    model = model_dic["tablewise"][str(shuffle_cols)][str(random_state)]
else:
    model = model_dic[f"{modifications}"][str(shuffle_cols)][str(random_state)]
print(f"Loaded model: {model}")  

# test model
total_true_labels = []
total_predicted_labels = []
for i in tqdm(range(0,len(test_dataset))):
    # if i > 0:
    #     break    
    response = openai.Completion.create(
        model= model,
        temperature = 0.8,
        max_tokens=600,
        prompt= test_dataset.iloc[i]["prompt"]
    )
    number_of_predictions = len(test_dataset.iloc[i]["completion"].split("\n"))
    true_labels = test_dataset.iloc[i]["completion"].split("\n")
    true_labels[0] = true_labels[0].replace(" ", "")
    
    predicted_labels = response["choices"][0]["text"].split("\n")[:number_of_predictions]
    predicted_labels[0] = predicted_labels[0].replace(" ", "")
    
    if number_of_predictions != len(predicted_labels):
        print(f"Number of labels should be {number_of_predictions} but is {len(predicted_labels)}")
        print(i)
        break
        
    total_true_labels.extend(true_labels)
    total_predicted_labels.extend(predicted_labels)
    
class_report = classification_report(
    total_true_labels, total_predicted_labels, output_dict=True)

with open(f"./results/GitTables_classification_report_{shuffle_cols}_{random_state}_0.7.json", "w") as f:
    json.dump(class_report, f)
    
with open(f"./results/GitTables_predictions_{shuffle_cols}_{random_state}_0.7.json", "w") as f:
    json.dump({"y_pred":total_predicted_labels, "y_true":total_true_labels}, f)

# Abbreviation buildings

In [ ]:
from dotenv import load_dotenv
load_dotenv(override=True)
from openai import OpenAI
from ast import literal_eval
import sys
sys.path.append("..")
import json
from data_loader.SportsDB_data_loader import get_all_numerical_semantic_types, get_all_textual_semantic_types

semantic_types = get_all_textual_semantic_types()+get_all_numerical_semantic_types()

client = OpenAI()

assistant = client.beta.assistants.retrieve("")

results = {}
for i, semantic_type in enumerate(semantic_types):
    # if i > 0:
    #     continue
    if len(semantic_type.split(".")) > 2:
        user_message = " ".join(semantic_type.split(".")[1:]).replace("_", " ")
        print(user_message)
    else:
        user_message = semantic_type
        print(user_message)

    thread = client.beta.threads.create(
        messages=[
            {
                "role": "user",
                "content": user_message
            }
        ]
    )

    run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
    )
    
    completed = False
    while completed == False:
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )
        if dict(run)["status"] == "completed":
            completed = True
            
    thread_messages = client.beta.threads.messages.list(thread.id)
    results[semantic_type] = literal_eval(dict(thread_messages.data[0])["content"][0].text.value)
    
    # save in each iteration the the results
    with open("SportsTables_semantic_type_abbreviations.json", "w") as f:
        json.dump(results, f)
